# Calibrating Camera

In [5]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import statistics as st
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline
np.random.seed(17)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx = 9 # Number of inside corners in x
ny = 6 # Number of inside corners in y)
    
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2) #Reshape in x,y coordinates

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
image_for_perspective_transform = glob.glob('test_images/straight_lines1.jpg')
images_for_calibration = glob.glob('camera_cal/calibration*.jpg')
img_size = mpimg.imread(images_for_calibration[0]).shape[1], mpimg.imread(images_for_calibration[0]).shape[0]

PointOne = (287, 660)
PointTwo = (557, 480)
PointThree = (731,480)
PointFour = (1010,660)



class Line:
    def __init__(self):
        self.old_curvatures = []
        self.horiz_distances = []
        self.poly_coeff_good_measurements = []

right_line = Line()
left_line = Line()
num_times_last_bad_pred = 3

def calibrate_camera(images, nx, ny):
# Step through the list and search for chessboard corners
    for image in images:
        img = mpimg.imread(image)

        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        # If found, add object points, image points

        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist

def get_perspective_matrix(image, mtx, dist):
    
    undistorted_img = cv2.undistort(image, mtx, dist, None, mtx)

    # As source points we get the outer four detected corners
    
    cv2.line(undistorted_img, PointOne, PointTwo, color=[255,0,0], thickness = 2)
    cv2.line(undistorted_img, PointTwo, PointThree, color=[255,0,0], thickness = 2)
    cv2.line(undistorted_img, PointThree, PointFour, color=[255,0,0], thickness = 2)
    cv2.line(undistorted_img, PointFour, PointOne, color=[255,0,0], thickness = 2)
    
    #plt.imshow(undistorted_img)
    #plt.figure()
    src_points = np.float32([PointOne, PointTwo, PointThree, PointFour])
    # As destination points we arbitrarily choose some points to be a nice fit
    dst_points = np.float32([[260, img_size[1]],[260, 0],
                                     [1020, 0],[1020, img_size[1]]
                                     ])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    inv_matrix = cv2.getPerspectiveTransform(dst_points, src_points )
    return matrix, inv_matrix

def warp_image(undistorted_img, matrix):

    warped_image = cv2.warpPerspective(undistorted_img, matrix, img_size)
    return warped_image

def threshold(image, s_thresh=(160, 220), sx_thresh=(45, 120)):
    sobel_kernel = 9
    img = np.copy(image)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    #
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Directional Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    thresh = (70,100)
    # Gradient magnitude
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    gradient_output = np.zeros_like(gradmag)
    gradient_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    
    dir_thresh = (0.7, 1.3)
    # Gradient of direction
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    directional_output =  np.zeros_like(absgraddir)
    directional_output[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
 
    # Combinate color and sobel thresholds:
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1) | (gradient_output == 1) | (directional_output == 1)] = 1
    
    return s_binary, sxbinary, combined_binary
        
def compare_thresholds(warped_image):
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(warped_image)
    ax1.set_title('Warped Image', fontsize=40)
    ax2.imshow(color_th, cmap='gray')
    ax2.set_title('S threshold', fontsize=40)
    ax3.imshow(sobel_th, cmap='gray')
    ax3.set_title('Sobel threshold', fontsize=40)
    ax4.imshow(combined_th, cmap='gray')
    ax4.set_title('Combined threshold', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
def detect_lane_pixels(binary_warped, image):
    global left_line
    global right_line
    global num_times_last_bad_pred
    if (num_times_last_bad_pred == 3):
        leftx, lefty, rightx, righty, out_img = sliding_windows(binary_warped)
        output_img, left_fit, right_fit, left_fitx, right_fitx = fit_polynomial(leftx, lefty, rightx, righty, out_img)
        left_line.poly_coeff_good_measurements.clear()
        left_line.old_curvatures.clear()
        right_line.poly_coeff_good_measurements.clear()
        right_line.old_curvatures.clear()
        
        left_curv, right_curv = measure_curvature_pixels(left_fitx, right_fitx, lefty, righty, output_img )
        #print("left curvature {} and right curvature {}".format(left_curv,right_curv))
        left_line.poly_coeff_good_measurements.append(left_fit)
        right_line.poly_coeff_good_measurements.append(right_fit)
        left_line.old_curvatures.append(left_curv)
        right_line.old_curvatures.append(right_curv)
        
        ## Change this number
        num_times_last_bad_pred = 0
        
        
    else: 
        output_img, left_fitx, right_fitx, leftx, rightx, left_fit, right_fit, left_curv, right_curv = search_around_poly(binary_warped)
    car_offset = measure_car_offset(leftx, rightx, binary_warped)
    image_with_lanes = draw_lanes(image, binary_warped, left_fitx, right_fitx)
    
    #plt.figure()
    #plt.imshow(output_img)
    return left_fit, right_fit, output_img, car_offset, image_with_lanes, left_curv, right_curv
    
def sliding_windows(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 3) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 3) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(leftx, lefty, rightx, righty, out_img):

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/900 # meters per pixel in x dimension
    # Fit a second order polynomial to each using `np.polyfit`
    #left_fit = np.polyfit(lefty *ym_per_pix, leftx *xm_per_pix, 2)
    #right_fit = np.polyfit(righty *ym_per_pix, rightx *xm_per_pix, 2)
    
    
    left_fit = np.polyfit(lefty , leftx , 2)
    right_fit = np.polyfit(righty , rightx , 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, out_img.shape[0]-1, out_img.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]
    #out_img[righty, rightx] = [0, 0, 255]
    # Plots the left and right polynomials on the lane lines
    #plt.figure()
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.imshow(out_img)
    return out_img, left_fit, right_fit, left_fitx, right_fitx

def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100
    global num_times_last_bad_pred
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_line.poly_coeff_good_measurements[-1][0]*(nonzeroy**2) + 
                    left_line.poly_coeff_good_measurements[-1][1]*nonzeroy + 
                    left_line.poly_coeff_good_measurements[-1][2] - margin)) & 
                    (nonzerox < (left_line.poly_coeff_good_measurements[-1][0]*(nonzeroy**2) + 
                    left_line.poly_coeff_good_measurements[-1][1]*nonzeroy + 
                    left_line.poly_coeff_good_measurements[-1][2] + margin)))
    right_lane_inds = ((nonzerox > (right_line.poly_coeff_good_measurements[-1][0]*(nonzeroy**2) + right_line.poly_coeff_good_measurements[-1][1]*nonzeroy + 
                    right_line.poly_coeff_good_measurements[-1][2] - margin)) & (nonzerox < (right_line.poly_coeff_good_measurements[-1][0]*(nonzeroy**2) + 
                    right_line.poly_coeff_good_measurements[-1][1]*nonzeroy + right_line.poly_coeff_good_measurements[-1][2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    out_img, left_fit, right_fit, left_fitx, right_fitx = fit_polynomial(leftx, lefty, rightx, righty, binary_warped)
    #return left_fitx, right_fitx
    left_curv, right_curv = measure_curvature_pixels(left_fitx, right_fitx, lefty, righty, binary_warped )
    # If new curvature value is bigger or lower than (100 + offset)% of curvatures median.
    offset = 15 
    
    #print(left_line.poly_coeff_good_measurements)
    # If not similar curvatures
    if (((st.median(left_line.old_curvatures) > (left_curv + (left_curv * offset)) or
        (st.median(left_line.old_curvatures) < left_curv - (left_curv * offset)))) 
        or ((st.median(right_line.old_curvatures) > (right_curv + (right_curv * offset))
        or (st.median(right_line.old_curvatures) < (right_curv - (right_curv * offset)))))):
        left_fit = [st.median(left_line.poly_coeff_good_measurements[:][0]), st.median(left_line.poly_coeff_good_measurements[:][1]), st.median(left_line.poly_coeff_good_measurements[:][2])]
        right_fit = [st.median(right_line.poly_coeff_good_measurements[:][0]), st.median(right_line.poly_coeff_good_measurements[:][1]), st.median(right_line.poly_coeff_good_measurements[:][2])]
        left_curv = st.median(left_line.old_curvatures)
        right_curv = st.median(right_line.old_curvatures)
        num_times_last_bad_pred += 1
        

    # if curvatures are similar, update data
    else:
        left_line.poly_coeff_good_measurements.append(left_fit)
        right_line.poly_coeff_good_measurements.append(right_fit)
        #left_fit = [st.median(left_line.poly_coeff_good_measurements[:][0]), st.median(left_line.poly_coeff_good_measurements[:][1]), st.median(left_line.poly_coeff_good_measurements[:][2])]
        #right_fit = [st.median(right_line.poly_coeff_good_measurements[:][0]), st.median(right_line.poly_coeff_good_measurements[:][1]), st.median(right_line.poly_coeff_good_measurements[:][2])]
        left_line.old_curvatures.append(left_curv)
        right_line.old_curvatures.append(right_curv)
        #left_curv = st.median(left_line.old_curvatures)
        #right_curv = st.median(right_line.old_curvatures)
        num_times_last_bad_pred = 0
        
        
        
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

    
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx, leftx, rightx, left_fit, right_fit, left_curv, right_curv

def measure_curvature_pixels(leftx, rightx, lefty, righty, image):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image

    ploty = np.linspace(0, image.shape[0] - 1, image.shape[0])

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/900 # meters per pixel in x dimension

    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y
    
    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty, leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty, rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 25/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/800 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    return left_curverad, right_curverad

def measure_car_offset(leftx, rightx, image, xm_per_pix=3.7/900):
    
    mid_imgx = image.shape[1]//2
        
    ## Car position with respect to the lane
    car_pos = (leftx[-1] + rightx[-1])/2
    
    ## Horizontal car offset 
    offsetx = (mid_imgx - car_pos) * xm_per_pix

    return offsetx

def draw_lanes(undist, warped_image, left_fitx, right_fitx):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_image).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    ploty = np.linspace(0, warped_image.shape[0]-1, warped_image.shape[0] )

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, inv_matrix, (warped_image.shape[1], warped_image.shape[0])) 
    # Combine the result with the original image
    return cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

def plot_stats_on_img(out_img, left_curv, right_curv, car_offset):

    # Display lane curvature
    cv2.putText(out_img, 'Left lane line curvature: {} m'.format(left_curv), 
                (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)
    cv2.putText(out_img, 'Right lane line curvature: {} m'.format(right_curv), 
                (60, 110), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)
    
    # Display car offset
    cv2.putText(out_img, 'Horizontal car offset: {:.2f} m'.format(car_offset), 
                (60, 160), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 5)
#    Syntax: cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])

    return out_img
    
mtx, dist = calibrate_camera(images_for_calibration, nx, ny)
matrix, inv_matrix  = get_perspective_matrix(mpimg.imread(image_for_perspective_transform[0]), mtx, dist)

def find_lanes(image):
    warped_image = warp_image(image, matrix)
    color_th, sobel_th, combined_th = threshold(warped_image)
    
    #compare_thresholds(warped_image)
    left_fit, right_fit, output_img, car_offset, image_with_lanes, left_curv, right_curv = detect_lane_pixels(combined_th, image)
    final_image = plot_stats_on_img(image_with_lanes, left_curv, right_curv, car_offset)

    #inverse_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    #binary_untransformed_img = cv2.warpPerspective(combined_th, inverse_matrix, img_size)
    return final_image
    
input_video = './project_video.mp4'
output_video = './project_video_solution.mp4'


clip1 = VideoFileClip(input_video)
# Process video frames with our 'process_image' function
video_clip = clip1.fl_image(find_lanes)
%time video_clip.write_videofile(output_video, audio=False)
HTML("""
<video width="920" height="440" controls>
  <source src="{0}">
</video>
""".format(output_video))
    

[MoviePy] >>>> Building video ./project_video_solution.mp4
[MoviePy] Writing video ./project_video_solution.mp4


100%|█████████▉| 1260/1261 [06:18<00:00,  3.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_solution.mp4 

CPU times: user 2min 50s, sys: 1min 12s, total: 4min 3s
Wall time: 6min 22s
